# Анализ и прогнозирование неоднородных временных рядов, БИ

## НИУ ВШЭ, 2022-23 учебный год

# Семинар 6. Временные ряды - анализ и модели

## Раздел 1. Временные ряды. Обработка, форматирование, визуализация

Временной ряд — собранный в разные моменты времени статистический материал о значении каких-либо параметров исследуемого процесса.

Скалярным временным рядом 

$$\{x_i\}_{i=1}^N$$

называется массив из $N$ чисел, представляющих собой значения некоторой измеренной (наблюдаемой) динамической переменной $x(t)$ с некоторым постоянным шагом $τ$ по времени,

 $$t_i = t_0 + (i − 1)τ : x_i = x(t_i), i = 1, . . . , N.$$  

Примеры: 
- объёмы продаж в торговых сетях
- объёмы потребления и цены электроэнергии
- остатки складских запасов
- рыночные цены
- дорожный трафик (прогнозирование пробок)

**Вопрос:** Какие задачи здесь возникают?

### Чтение данных

В рамках этого ноутбука будем работать с данными по продажам в Индии в различных городах и штатах

Для начала визуализируем наши данные и попробуем сформулировать новые гипотезы о задаче, которую хотим решить.

Загрузим данные и посмотрим на структуру данных

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import statsmodels.api as sm
import statsmodels.formula.api as smf
from statsmodels.tsa.stattools import adfuller
%matplotlib inline

pd.options.display.float_format = '{:.2f}'.format

df = pd.read_csv('MarketArrivals_Ind.csv')

FileNotFoundError: [Errno 2] No such file or directory: 'MarketArrivals_Ind.csv'

### Анализ структуры данных

In [ ]:
df.shape

In [ ]:
df.head()

#### Cтруктура данных

В нашем датафрейме 10 столбцов.

В датафрейме следующие типы данных:
- **CATEGORICAL**: state, city, market
- **TIME INTERVAL**: month, year, date
- **QUANTITATIVE**: quantity, priceMin, priceMax, priceModal

Посмотрим какие типы данных pandas определил при чтении данных.

In [ ]:
# Выведем типы данных
df.dtypes

Видим, что количественные столбцы отображаться как целочисленные и категориальные как объекты(строки), что отлично.
Однако, столбцы `date` прочитаны как объект, а не даты(время). Исправим это.

In [ ]:
df.date = pd.DatetimeIndex(df.date)

In [ ]:
df.shape

In [ ]:
df.dtypes

In [ ]:
df.head()

### Задание 1. Определите, насколько большой был рынок в Бангалоре по сравнению с другими городами в 2010 году?

Ответим на этот вопрос, используя данные 2010го года.

In [ ]:
df2010 = df[df.year == 2010]

In [ ]:
df2010.head()

In [ ]:
df2010Bang = df[(df.year == 2010) & (df.city == 'BANGALORE')]

In [ ]:
df2010Bang.head()

In [ ]:
# Считаем суммы по quantity
df2010City = df2010.groupby(['city']).quantity.sum()

In [ ]:
df2010City.head()

In [ ]:
type(df2010City)

In [ ]:
# Создаем dataframe
df2010City = df2010.groupby(['city'], as_index=False).quantity.sum()

In [ ]:
df2010City.head()

In [ ]:
# Cортируем df по Quantity
df2010City = df2010City.sort_values(by = "quantity", ascending = False)
df2010City.head()


Визуализируем наши выводы

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
plt.style.use('ggplot')
plt.rcParams['figure.figsize'] = (15, 15)
df2010City.plot(kind ="barh", x = 'city', y = 'quantity', colormap='summer')

### Задание 2. Найти штат с наибольшими продажами в 2015 году

In [ ]:
### your code here

### Задание 3. Вывести график кол-ва продаж в штате MS по всем городам в 2015 году

In [ ]:
### your code here

### Задание 4. Растет ли разброс цен в Бангалоре с годами?

In [ ]:
df.head()

In [ ]:
dfBang = df[df.city == 'BANGALORE']

In [ ]:
dfBang.head()

In [ ]:
dfBang.describe()

#### Установим Index как date

In [ ]:
dfBang.head()

In [ ]:
dfBang.index

In [ ]:
# Упорядочим по датам
dfBang = dfBang.sort_values(by = "date")
dfBang.head()

In [ ]:
# Установим index
dfBang.index = pd.PeriodIndex(dfBang.date, freq='M')

In [ ]:
dfBang.head()

#### Нарисуем графики

In [ ]:
dfBang.priceMod.plot()

Мы получили временной ряд !

In [ ]:
dfBang.plot(kind = "line", y = ['priceMin', 'priceMod', 'priceMax'])

Мы получили сразу 3 временных ряда на одних осях !

Для расчета изменения создадим новый атрибут с разницей priceMax и priceMin  

In [ ]:
dfBang['priceDiff'] = dfBang['priceMax'] - dfBang['priceMin']

In [ ]:
dfBang.head()

Выведем график priceDiff

In [ ]:
dfBang.plot(kind = 'line', y = 'priceDiff')

## Раздел 2. Работа со сводными таблицами (pivot table)

Возможность создавать сводные таблицы присутствует в электронных таблицах и других программах, оперирующих табличными данными.
Сводная таблица принимает на входе данные из отдельных столбцов и группирует их, формируя двумерную таблицу, реализующую многомерное обобщение данных.
Чтобы ощутить разницу между сводной таблицей и операцией GroupBy, можно представить себе сводную таблицу, как многомерный вариант агрегации посредством GroupBy. То есть данные разделяются, преобразуются и объединяются, но при этом разделение и объединение осуществляются не по одномерному индексу, а по двумерной сетке.


![](https://pandas.pydata.org/pandas-docs/stable/_images/reshaping_pivot.png)


In [ ]:
# Создадим новый столбец для значения Месяц
dfBang['monthVal'] = pd.DatetimeIndex(dfBang['date']).month

In [ ]:
dfBang.head()

Cоздадим сводную таблицу по priceDiff (строки - monthval, столбцы - year)

In [ ]:
dfBangPivot = pd.pivot_table(dfBang, values = "priceDiff", 
                             columns = "year", index = "monthVal")

In [ ]:
dfBangPivot

In [ ]:
plt.style.use('ggplot')
dfBangPivot.plot()

### Задание 5. Вывести сводную таблицу с quantity по штатам и годам

In [ ]:
### your code here

## Раздел 3. Прогнозирование временного ряда

### План
#### 3. Прогнозирование временного ряда
#### 3.1 Прогнозирование средним (Mean Constant Model)
#### 3.2. Линейная модель (Linear Trend Model)
#### 3.3. Линейная модель с регрессией (Linear Regression Model)
#### 4. Выделение тренда и сезонности
#### 5. Стационарность ряда
#### 6. ARIMA

### Подготовка данных

In [ ]:
# Отсортируем и добавим date в индекс
df.index = pd.PeriodIndex(df.date, freq='M')
df.index.rename('date_ind', inplace = True)
df = df.sort_values(by = "date")

In [ ]:
df.head()

Будем работать с данными в городе Бангалор

In [ ]:
dfBang = df.loc[df.city == "BANGALORE"].copy()

In [ ]:
dfBang.head()

In [ ]:
# Удалим лишние столбцы
dfBang = dfBang.drop(["market", "month", "year", "state", "city", "priceMin", "priceMax"], axis = 1)

In [ ]:
dfBang.shape

In [ ]:
dfBang.head()

In [ ]:
dfBang.priceMod.plot()

In [ ]:
dfBang.quantity.plot()

### Логарифмическое преобразование данных
Логарифмирование помогает стабилизировать разброс значений.

In [ ]:
dfBang.priceMod.plot(kind = "hist", bins = 30)

При визуализации можем заметить, что есть данные, далеко отстающие от среднего значения, что может искажать модель регрессии. Наша задача минимизировать влияние таких данных. Желательно не исключая их.  
В этом случае самый распространенный вариант действия - логарифмирование переменной.

In [ ]:
dfBang['priceModLog'] = np.log(dfBang.priceMod)
dfBang.head()

In [ ]:
dfBang.priceModLog.plot(kind = "hist", bins = 30)

In [ ]:
dfBang.priceModLog.plot()

## 3.1. Прогнозирование средним 

In [ ]:
model_mean_pred = dfBang.priceModLog.mean()

In [ ]:
# Запишем это значение в priceMean
dfBang["priceMean"] = np.exp(model_mean_pred)

In [ ]:
dfBang.plot(kind="line", x= 'date', y = ["priceMod", "priceMean"])

**Как мерить качество?**

Чтобы обучать регрессионные модели, нужно определиться, как именно измеряется качество предсказаний.   
Будем использовать метрику RMSE (Root Mean Squared Error) - корень среднего квадрата отклонения.

$$RMSE =  \sqrt{ \Sigma (\hat{y} - y)^2/n} $$ , 
где $\hat{y}$ это предсказанное значение y

In [ ]:
def RMSE(predicted, actual):
    mse = (predicted - actual)**2
    rmse = np.sqrt(mse.sum()/mse.count())
    return rmse

In [ ]:
model_mean_RMSE = RMSE(dfBang.priceMean, dfBang.priceMod)
model_mean_RMSE

In [ ]:
dfBangResults = pd.DataFrame(columns = ["Model", "RMSE"])
dfBangResults.head()

In [ ]:
dfBangResults.loc[0,"Model"] = "Mean"
dfBangResults.loc[0,"RMSE"] = model_mean_RMSE
dfBangResults.head()

### Задание 6. Постройте модель средних для величины quantity

In [ ]:
## your code here

## 3.2. Линейная модель

Построим линейную зависиммость между priceModLog and time.   
Уравнение регрессии строится с помощью метода наимменьших квадратов.

In [ ]:
dfBang.head()

In [ ]:
dfBang.dtypes

In [ ]:
dfBang.date.min()

In [ ]:
dfBang["timeIndex"] = dfBang.date - dfBang.date.min()

In [ ]:
dfBang.dtypes

In [ ]:
dfBang["timeIndex"] =  dfBang["timeIndex"]/np.timedelta64(1, 'M')

In [ ]:
dfBang["timeIndex"] = dfBang["timeIndex"].round(0).astype(int)

In [ ]:
dfBang.timeIndex.tail()

In [ ]:
dfBang.head()

In [ ]:
model_linear = smf.ols('priceModLog ~ timeIndex', data = dfBang).fit()

In [ ]:
model_linear.summary()

#### Выведем параметры нашей линейной модели

In [ ]:
model_linear.params

In [ ]:
c = model_linear.params[0]
c

In [ ]:
m = model_linear.params[1]
m

In [ ]:
model_linear_pred = model_linear.predict()

In [ ]:
model_linear_pred

In [ ]:
dfBang["priceLinear"] = np.exp(model_linear_pred)

In [ ]:
dfBang.plot(kind="line", x="timeIndex", y = "priceModLog")
plt.plot(dfBang.timeIndex,model_linear_pred, '-')

In [ ]:
model_linear.resid.plot(kind = "bar")

Посчитаем ошибку RMSE на линейной модели:

In [ ]:
model_linear_RMSE = RMSE(dfBang.priceLinear, dfBang.priceMod)
model_linear_RMSE

In [ ]:
dfBangResults.loc[1,"Model"] = "Linear"
dfBangResults.loc[1,"RMSE"] = model_linear_RMSE
dfBangResults.head()

### Задание 7. Постройте линейную модель для величины quantity

In [ ]:
### your code here

## 3.3 Линейная модель с регрессией

In [ ]:
dfBang.tail()

In [ ]:
## Построим регрессию, используя несколько переменных timeIndex и np.log(quantity)
model_linear_quantity = smf.ols('priceModLog ~ timeIndex + np.log(quantity)', data = dfBang).fit()

In [ ]:
model_linear_quantity.summary()

In [ ]:
dfBang.head()

In [ ]:
dfBang.tail()

In [ ]:
model_linear_quantity.predict().shape

In [ ]:
dfBang["priceLinearQuantity"] = np.exp(model_linear_quantity.predict())

In [ ]:
model_linear_q_pred = model_linear_quantity.predict()

In [ ]:
dfBang.plot(kind = "line", x="timeIndex", y = "quantity")
dfBang.plot(kind="line", x="timeIndex", y = ["priceMod", "priceMean", 
                                             "priceLinear", "priceLinearQuantity"])

### Прогнозирование в будущее

Также мы можем обучить нашу модель на части выборки, чтобы осуществить прогноз в будущее. Для оставшихся данных в начале сделаем прогноз, затем оценим, насколько он был точный

In [ ]:
dfBang.tail()

In [ ]:
model_linear_quantity = smf.ols('priceModLog ~ timeIndex + np.log(quantity)', data = dfBang.iloc[:-15, :]).fit()
model_linear_quantity.predict(dfBang.iloc[-15:, :])

In [ ]:
prediction = np.exp(model_linear_quantity.predict(dfBang.iloc[-15:, :]).values)

In [ ]:
plt.plot(dfBang.date[:-15], dfBang['priceMod'][:-15])
plt.plot(dfBang.date[-15:], prediction)

In [ ]:
plt.plot(dfBang.date, dfBang['priceMod'])
plt.plot(dfBang.date[-15:], prediction)

### Задание 8. Постройте линейную модель с регрессией для величины quantity

In [ ]:
### your code here

## 4. Выделение тренда и сезонности

При работе с временными рядами выделяют два действия:
- Дифференцирование - расчет разницы с определенным временным лагом
- Декомпозиция - моделирование как тренда, так и сезонности и удаление их из модели.

Предположим, что у вас имеются ежемесячные данные о пассажиропотоке на международных авиалиниях.  
Объем пассажиропотока имеет во времени возрастающий линейный тренд, и в ряде имеется ежегодно повторяющаяся закономерность - сезонность (большинство перевозок приходится на летние месяцы, кроме того, имеется пик меньшей высоты в районе декабрьских каникул). 
Цель декомпозиции как раз и состоит в том, чтобы отделить эти компоненты, то есть разложить ряд на составляющую тренда, сезонную компоненту и оставшуюся нерегулярную составляющую. 

То есть каждый уровень временного ряда формируется под воздействием большого числа факторов, которые условно можно подразделить на три группы:
- факторы, формирующие тенденцию ряда
- факторы, формирующие циклические колебания ряда
- случайные факторы  

Рассмотрим воздействие каждого фактора на временной ряд в отдельности.  
Большинство временных рядов экономических показателей имеют тенденцию, 
характеризующую совокупное долговременное воздействие множества факторов на динамику изучаемого показателя.  
Все эти факторы, взятые в отдельности, могут оказывать разнонаправленное воздействие на исследуемый показатель. Однако в совокупности они формируют его возрастающую или убывающую тенденцию

В большинстве случаев фактический уровень временного ряда можно представить как сумму или произведение трендовой, циклической и случайной компонент. Модель, в которой временной ряд представлен как сумма перечисленных компонент, называется **аддитивной** моделью временного ряда. Модель, в которой временной ряд представлен как произведение перечисленных компонент, называется **мультипликативной** моделью временного ряда. 

**Аддитивная модель** 
$${Y_t} = t (trend) + s (seasonality) + r (residual)$$

![](https://miro.medium.com/max/1360/0*JWWglnH4RBR-SbxC.png)

**Мультипликативная модель** 
$${Y_t} = t (trend) * s (seasonality) * r (residual)$$

![](https://anomaly.io/wp-content/uploads/2015/12/multiplicative-model.png)

Как видим, отличие мультипликативной модели от аддитивной состоит в том, 
что в мультипликативной модели сезонная и случайная составляющие определены в виде относительных величин (коэффициентов), 
а в аддитивной модели – в виде абсолютных величин.   
Эти модели в практических расчетах дадут близкие результаты, если амплитуда колебаний уровней ряда слабо изменяется во времени

Какие примеры аддитивных и мультипликаативных рядов из жизни вы знаете?

### Декомпозиция 
Декомпозируем наш временной ряд на тренд и сезонность

In [ ]:
from statsmodels.tsa.seasonal import seasonal_decompose

In [ ]:
dfBang.index = dfBang.index.to_timestamp()

In [ ]:
dfBang.head()

In [ ]:
decomposition = seasonal_decompose(dfBang.priceModLog, model = "additive")

In [ ]:
decomposition.plot()

In [ ]:
trend = decomposition.trend
seasonal = decomposition.seasonal
residual = decomposition.resid

In [ ]:
# Построим ряд по тренду и сезонности
dfBang["priceDecomp"] = np.exp(trend + seasonal)

In [ ]:
# Расчет RMSE
model_Decomp_RMSE = RMSE(dfBang.priceDecomp, dfBang.priceMod)
model_Decomp_RMSE

In [ ]:
dfBang.plot(kind="line", x="timeIndex", y = ["priceMod", "priceMean", "priceLinear", "priceLinearQuantity", "priceDecomp"])

In [ ]:
dfBangResults

In [ ]:
dfBangResults.loc[3,"Model"] = "priceDecomp"
dfBangResults.loc[3,"RMSE"] = model_Decomp_RMSE
dfBangResults.head()

### Задание 9. Декомпозируйте ряд остатков quantity

In [ ]:
### your code here

## 5. Стационарность ряда

Перед тем, как перейти к моделированию, стоит сказать о таком важном свойстве временного ряда, как стационарность.  
Под стационарностью понимают свойство процесса не менять своих статистических характеристик с течением времени, а именно постоянство матожидания, постоянство дисперсии и независимость ковариационной функции от времени (должна зависеть только от расстояния между наблюдениями). Наглядно можно посмотреть на эти свойства на картинках:

![](https://habrastorage.org/files/20c/9d8/a63/20c9d8a633ec436f91dccd4aedcc6940.png)  
Временной ряд справа не является стационарным, так как его матожидание со временем растёт

![](https://habrastorage.org/files/b88/eec/a67/b88eeca676d642449cab135273fd5a95.png)  
Здесь не повезло с дисперсией — разброс значений ряда существенно варьируется в зависимости от периода

![](https://habrastorage.org/files/2f6/1ee/cb2/2f61eecb20714352840748b826e38680.png)  
На последнем графике видно, что значения ряда внезапно становятся ближе друг ко другу, образуя некоторый кластер, а в результате получаем непостоянство ковариаций


Почему стационарность так важна?  
По стационарному ряду просто строить прогноз, так как мы полагаем, что его будущие статистические характеристики не будут отличаться от наблюдаемых текущих. Большинство моделей временных рядов так или иначе моделируют и предсказывают эти характеристики (например, матожидание или дисперсию), поэтому в случае нестационарности исходного ряда предсказания окажутся неверными. К сожалению, большинство временных рядов, с которыми приходится сталкиваться за пределыми учебных материалов, стационарными не являются, но с этим можно (и нужно) бороться.

In [ ]:
test = sm.tsa.adfuller(dfBang.priceMod)
print('adf: ', test[0])
print('p-value: ', test[1])
print('Critical values: ', test[4])
if test[0]> test[4]['5%']: 
    print('Есть единичные корни, ряд не стационарен')
else:
    print('Единичных корней нет, ряд стационарен')

# p-value > 0.05: Fail to reject the null hypothesis (H0), the data has a unit root and is non-stationary.
# p-value <= 0.05: Reject the null hypothesis (H0), the data does not have a unit root and is stationary.

### Расчет лагов
Представим временной ряд в виде последовательности разницы между значениями.  
Рассчитаем лаги, выведем их на графике

In [ ]:
# Рассчитаем лаги
dfBang["priceModLogShift1"] = dfBang.priceModLog.shift()

In [ ]:
# Выведем на графике priceModLog и priceModLogShift1
dfBang.plot(kind= "scatter", y = "priceModLog", x = "priceModLogShift1", s = 50)

In [ ]:
dfBang["priceModLogDiff"] = dfBang.priceModLog - dfBang.priceModLogShift1

In [ ]:
dfBang.priceModLogDiff.plot()

## 6. Построение модели АRIMA

Для моделирования будем использовать модель ARIMA, построенную для ряда первых разностей.  
Итак, чтобы построить модель нам нужно знать ее порядок, состоящий из 2-х параметров:  
- p — порядок компоненты AR
- d — порядок интегрированного ряда (порядок разности временного ряда)
- q — порядок компонетны MA  


Параметр d равет 1, осталось определить p и q. Для их определения нам надо изучить авторкорреляционную (ACF) и частично автокорреляционную (PACF) функции для ряда первых разностей.
ACF поможет нам определить q, т. к. по ее коррелограмме можно определить количество автокорреляционных коэффициентов сильно отличных от 0 в модели MA
PACF поможет нам определить p, т. к. по ее коррелограмме можно определить максимальный номер коэффициента сильно отличный от 0 в модели AR.  

Чтобы построить соответствующие коррелограммы, в пакете statsmodels имеются следующие функции: acf и pacf.  
Можем построить графики ACF и PACF, у которых по оси X откладываются номера лагов, а по оси Y значения соответствующих функций. Нужно отметить, что количество лагов в функциях и определяет число значимых коэффициентов.

In [ ]:
ts = dfBang.priceModLog
ts_diff = dfBang.priceModLogDiff
ts_diff.dropna(inplace = True)

In [ ]:
ts = dfBang.priceModLog
ts_diff = dfBang.priceModLogDiff
ts_diff.dropna(inplace = True)

In [ ]:
# ACF и PACF 
from statsmodels.tsa.stattools import acf, pacf

In [ ]:
lag_acf = acf(ts_diff, nlags=20)

In [ ]:
lag_acf

In [ ]:
ACF = pd.Series(lag_acf)

In [ ]:
ACF

In [ ]:
ACF.plot(kind = "bar")

In [ ]:
lag_pacf = pacf(ts_diff, nlags=20, method='ols')

In [ ]:
PACF = pd.Series(lag_pacf)

In [ ]:
PACF.plot(kind = "bar")

## Построим модель ARIMA 

In [ ]:
from statsmodels.tsa.arima.model import ARIMA

In [ ]:
ts_diff.head()

In [ ]:
# ARIMA Model (1,0,1)
model_AR1MA = ARIMA(ts_diff, order=(1,0,1))

In [ ]:
results_ARIMA = model_AR1MA.fit()

In [ ]:
results_ARIMA.fittedvalues.head()

In [ ]:
ts_diff.plot()
results_ARIMA.fittedvalues.plot()

In [ ]:
ts_diff.sum()

In [ ]:
predictions_ARIMA_diff = pd.Series(results_ARIMA.fittedvalues, copy=True)
predictions_ARIMA_diff.tail()

In [ ]:
predictions_ARIMA_diff.sum()

In [ ]:
predictions_ARIMA_diff_cumsum = predictions_ARIMA_diff.cumsum()
predictions_ARIMA_diff_cumsum.tail()

In [ ]:
ts.iloc[0]

In [ ]:
predictions_ARIMA_log = pd.Series(ts.iloc[0], index=ts.index)
predictions_ARIMA_log = predictions_ARIMA_log.add(predictions_ARIMA_diff_cumsum, fill_value=0)
predictions_ARIMA_log.tail()

In [ ]:
dfBang['priceARIMA'] = np.exp(predictions_ARIMA_log)

In [ ]:
dfBang.plot(kind="line", x="timeIndex", y = ["priceMod", "priceARIMA"])

In [ ]:
dfBang.plot(kind="line", x="timeIndex", y = ["priceMod", "priceMean", "priceLinear", 
                                             "priceLinearQuantity", "priceARIMA"])

In [ ]:
# RMSE
model_arima_RMSE = RMSE(dfBang.priceARIMA, dfBang.priceMod)
model_arima_RMSE

Обычно подбор параметров для ARIMA — сложный и трудоемкий процесс.  
23-го февраля 2017 года команда Core Data Science из Facebook выпустила новую библиотеку для работы с временными рядами — Prophet. Попробуйте применить новую библиотеку и посмотрите на удобство в использовании и качество предсказания.

### Вывод

На семинаре мы ознакомились с разными подходами к анализу и прогнозированию временных рядов.  
Была рассмотрена интегрированная модель авторегрессии – скользящего среднего (ARIMA), как базовая модель прогнозирования.  
Как вы уже убедились, серебряной пули для решения такого рода задач пока нет.  
Методы, разработанные в 60-е годы прошлого века, (а некоторые и в начале 19-го), по-прежнему пользуются популярностью наравне с неразобранными в рамках данной статьи LSTM или RNN. Отчасти это связано с тем, что задача прогнозирования, как и любая другая задача, возникающая в процессе работы с данными — во многом творческая и уж точно исследовательская. Несмотря на обилие формальных метрик качества и способов оценки параметров, для каждого временного ряда часто приходится подбирать и пробовать что-то своё. Не последнюю роль играет и баланс между качеством и трудозатратами.